<a href="https://colab.research.google.com/github/bkansal/Simple_QRcode/blob/main/QRcode_standardlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import math
def create_qr_code(data, version, matrix_size):
    # Define the QR code parameters
    version = 1
    error_correction = 'L'
    box_size = 5
    border = 4

    # Encode the data into a binary string
    binary_data = ''.join(format(ord(char), '08b') for char in data)

    # Determine the size of the QR code matrix
    matrix_size = 4 * version + 17

    # Initialize the QR code matrix with all zeros
    qr_matrix = [[0] * matrix_size for _ in range(matrix_size)]

    # Add function patterns (finder patterns, separator, timing patterns)
    add_function_patterns(qr_matrix)

    # Add data to the QR code matrix
    add_data_to_matrix(binary_data, qr_matrix)

    # Add error correction codes
    add_error_correction(qr_matrix, error_correction)

    # Display the QR code matrix
    display_qr_code(qr_matrix)

def add_function_patterns(matrix):
    # Add finder patterns
    finder_pattern = [
        [1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0, 1],
        [1, 0, 1, 1, 1, 0, 1],
        [1, 0, 1, 1, 1, 0, 1],
        [1, 0, 1, 1, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1]
    ]

    # Add the finder pattern to the top-left corner
    for i in range(7):
        for j in range(7):
            matrix[i][j] = finder_pattern[i][j]
            matrix[i][matrix_size - 7 + j] = finder_pattern[i][j]
            matrix[matrix_size - 7 + i][j] = finder_pattern[i][j]

    # Add separator
    separator = [0] * matrix_size
    matrix[7] = separator
    matrix[matrix_size - 8] = separator
    for i in range(matrix_size):
        matrix[i][7] = 0
        matrix[i][matrix_size - 8] = 0

    # Add timing patterns
    for i in range(8, matrix_size - 8):
        matrix[i][6] = i % 2  # Vertical timing pattern
        matrix[6][i] = i % 2  # Horizontal timing pattern

def add_data_to_matrix(data, matrix):
    row, col, direction = matrix_size - 1, matrix_size - 1, -1

    for bit in data:
        matrix[row][col] = int(bit)
        col += direction

        if col < 0 or col >= matrix_size:
            row += direction
            direction = -direction
            col += direction

def multiply_polynomials(poly1, poly2):
    """
    Multiply two polynomials.
    """
    result = [0] * (len(poly1) + len(poly2) - 1)

    for i in range(len(poly1)):
        for j in range(len(poly2)):
            result[i + j] ^= poly1[i] * poly2[j]
    return result

def add_error_correction(original_data, error_correction):
    block_size = len(original_data)
    #print(original_data)
    # Define the size of the error correction block based on the error correction level
    if error_correction == 'L':
        ec_block_size = 2
    elif error_correction == 'M':
        ec_block_size = 4
    elif error_correction == 'Q':
        ec_block_size = 6
    elif error_correction == 'H':
        ec_block_size = 8
    else:
        raise ValueError("Invalid error correction level")

    # Calculate the total block size including error correction
    total_block_size = block_size + ec_block_size
    #print(list(original_data))

    # Initialize the coefficients for the original data polynomial
    original_coefficients = list(original_data) + [0] * ec_block_size

    # Initialize the coefficients for the Reed-Solomon code polynomial
    rs_coefficients = [1] * ec_block_size
    #print(rs_coefficients)
     # Generate Reed-Solomon code coefficients using the original data coefficients
    for i in range(ec_block_size):
        #print(rs_coefficients, " : ", [1, pow(2, i, 0x11b)])
        rs_coefficients = multiply_polynomials(rs_coefficients, [1, pow(2, i)])

    # Create the final encoded data by appending the Reed-Solomon code coefficients
    encoded_data = original_data + rs_coefficients
    #print(original_data)
    return encoded_data
    # TODO: Implement error correction codes based on version and error_correction level
    #pass

def display_qr_code(matrix):
    for row in matrix:
        print(' '.join(map(lambda x: '■' if x else '□', row)))

# Example usage
version = 1
matrix_size = 4*version + 17
data_to_encode = "Hello, World!"
create_qr_code(data_to_encode,version, matrix_size)


■ ■ ■ ■ ■ ■ ■ □ □ □ □ □ □ □ ■ ■ ■ ■ ■ ■ ■
■ □ □ □ □ □ ■ □ □ □ □ □ □ □ ■ □ □ □ □ □ ■
■ □ ■ ■ ■ □ ■ □ □ □ □ □ □ □ ■ □ ■ ■ ■ □ ■
■ □ ■ ■ ■ □ ■ □ □ □ □ □ □ □ ■ □ ■ ■ ■ □ ■
■ □ ■ ■ ■ □ ■ □ □ □ □ □ □ □ ■ □ ■ ■ ■ □ ■
■ □ □ □ □ □ ■ □ □ □ □ □ □ □ ■ □ □ □ □ □ ■
■ ■ ■ ■ ■ ■ ■ □ □ ■ □ ■ □ □ ■ ■ ■ ■ ■ ■ ■
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
■ ■ ■ ■ ■ ■ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ □
■ □ □ □ □ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ □
■ □ ■ ■ ■ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ □
■ □ ■ ■ ■ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ □
■ □ ■ ■ ■ □ ■ □ □ □ □ □ □ □ □ □ □ □ □ □ □
■ □ ■ ■ □ ■ ■ ■ ■ □ ■ ■ ■ □ □ ■ □ □ ■ ■ □
■ ■ □ □ □ □ ■ □ □ □ □ ■ □ □ ■ ■ □ □ □ ■ ■
